#### Seperating the videos with Yawn and talking into a seperate folder

In [97]:
import os
import cv2
import shutil
os.makedirs('YAWN',exist_ok=True)
subfolder=['images','videos']
for folder in subfolder:
    os.makedirs(f'YAWN/{folder}',exist_ok=True)
    if folder=='videos':
        os.makedirs(f'YAWN/{folder}/no_yawn',exist_ok=True)
        os.makedirs(f'YAWN/{folder}/yawn',exist_ok=True)
HOME='YawDD dataset'
dir_list=[dir for dir in os.listdir(HOME) if os.path.isdir(os.path.join(HOME,dir))]
for dir in dir_list:
    dir_path=os.path.join(HOME,dir)
    if dir=='Dash':
        subfolder_list=os.listdir(dir_path)
        for subfolder in subfolder_list:
            subfolder_path=os.path.join(dir_path,subfolder)
            video_list=os.listdir(subfolder_path)
            for video in video_list:
                video_path=os.path.join(subfolder_path,video)
                save_folder_path='YAWN/videos/yawn'
                dst_path=os.path.join(save_folder_path,video)
                # shutil.copy(src, dst)
                shutil.copy(video_path,dst_path)

    else:
        subfolder_list=os.listdir(dir_path)
        for subfolder in subfolder_list:
            subfolder_path=os.path.join(dir_path,subfolder)
            video_list=os.listdir(subfolder_path)
            for video in video_list:
                video_path=os.path.join(subfolder_path,video)
                if '-Yawning' in video:
                    save_folder_path='YAWN/videos/yawn'
                    dst_video_name=f'{dir}_{video}'
                    dst_path=os.path.join(save_folder_path,dst_video_name)
                    # dst_path=os.path.join(save_folder_path,video)
                    # shutil.copy(src, dst)
                    shutil.copy(video_path,dst_path)
                elif '-Talking' in video:
                    save_folder_path='YAWN/videos/no_yawn'
                    dst_video_name=f'{dir}_{video}'
                    dst_path=os.path.join(save_folder_path,dst_video_name)
                    # shutil.copy(src, dst)
                    shutil.copy(video_path,dst_path)
                # print(video)

#### Checking if the YOLO model detects sunglasses as closed eyes

In [ ]:
import cv2
from ultralytics import YOLO
model=YOLO('eye_detect.pt')
class_dict=model.names
HOME='YAWN/videos'
subfolder_list=os.listdir(HOME)
sunglasses_list=[]
for subfolder in subfolder_list:
    subfolder_path=os.path.join(HOME,subfolder)
    video_list=os.listdir(subfolder_path)
    for video in video_list:
        video_path=os.path.join(subfolder_path,video)
        cap=cv2.VideoCapture(video_path)
        ret,frame=cap.read()
        results=model.predict(frame,conf=0.7)
        result=results[0]
        for box in result.boxes:
            class_name=class_dict[box.cls.cpu().numpy().astype(int)[0]]
            if class_name=='drowsy':
                shutil.copy(video_path,video)
        
       

#### Detecting yawning frames using SPIGA

In [1]:
import numpy as np
def yawn_aspect_ratio_SPIGA(landmarks_int):
    """
    AR Formula= AR=||P2-P6||+||P3-P5||/2*(||P2-P4||)
    """
    
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    denominator_YAWN= np.linalg.norm(landmarks_int[91] - landmarks_int[88])  # Left Eye
    
    # ||P3-P5|| left
    yawn_89_95_dist=np.linalg.norm(landmarks_int[89] - landmarks_int[95])
   
    #||P2-P6|| left
    yawn_90_93_dist=np.linalg.norm(landmarks_int[90] - landmarks_int[93])
    
    yawn_AR=((yawn_89_95_dist+yawn_90_93_dist)/(2*denominator_YAWN))
    
    return yawn_AR

#### Test code of SPTGA for a single video

In [ ]:
import os
from ultralytics import YOLO
import numpy as np
import cv2

from spiga.demo.visualize.plotter import Plotter
from spiga.inference.config import ModelConfig
from spiga.inference.framework import SPIGAFramework

yawn_number=3
no_yawn_number=2


os.makedirs('test',exist_ok=True)
subfolder=['images','labels']
for folder in subfolder:
    os.makedirs(f'test/{folder}',exist_ok=True)

save_image_folder='test/images'
save_label_folder='test/labels'

# path_yawn=r'/home/cgi/drive/drowsy_att/YawDD.rar/YAWN/videos/yawn/Mirror_1-FemaleNoGlasses-Yawning.avi'
# path_yawn=r'/home/cgi/drive/drowsy_att/YawDD.rar/YAWN/videos/yawn/5-MaleGlasses.avi'
path_yawn=r'11-FemaleGlasses.avi.avi'
cap=cv2.VideoCapture(path_yawn)
model_yolo_path=r'/home/cgi/drive/drowsy_att/YawDD.rar/yolov8n-face.pt'
model_eyes_path=r'/home/cgi/drive/drowsy_att/YawDD.rar/eye_detect.pt'
model_yolo=YOLO(model_yolo_path)
model_eyes=YOLO(model_eyes_path)

eyes_dict=model_eyes.names
dataset='wflw'
processor=SPIGAFramework(ModelConfig(dataset))
plotter=Plotter()

while cap.isOpened():
    ret,frame=cap.read()
    if not ret:
        print("End of Video")
        break
    frame2=frame.copy()
    results=model_yolo.predict(frame,conf=0.6)
    result=results[0]
    # Saving image file into the images folder

    for box in result.boxes:
        x0,y0,x1,y1=box.xyxy.cpu().numpy().astype(int)[0]
        cv2.rectangle(frame,(x0,y0),(x1,y1),(255,0,0),2)
        w=abs(x1-x0)
        h=abs(y1-y0)
        bbox=[x0,y0,w,h]
        features=processor.inference(frame,[bbox])
        # Prepare variables
        landmarks=np.array(features['landmarks'][0])
        headpose=np.array(features['headpose'][0])
        x1=round(headpose[0],3)
        x2=round(headpose[1],3)
        x3=round(headpose[2],3)
       
        # # print(headpose)
        frame=plotter.landmarks.draw_landmarks(frame,landmarks,thick=1)
        frame=plotter.hpose.draw_headpose(frame,[x0,y0,x0+w,y0+h],headpose[:3],headpose[3:],euler=True)
        landmarks_int=[landmark.astype('int') for landmark in landmarks]
        yawn_AR=yawn_aspect_ratio_SPIGA(landmarks_int)
        if 'Mirror' in path_yawn:
            if yawn_AR>0.7:
                eyes_results=model_eyes.predict(frame2)
                eyes_result=eyes_results[0]
                video_name=path_yawn.split('/')[-1]
                frame_number=cap.get(cv2.CAP_PROP_POS_FRAMES)
                common_name=f'{video_name}_{frame_number}'
                label_name=common_name+'.txt'
                label_path=os.path.join(save_label_folder,label_name)

                for box_eyes in eyes_result.boxes:
                    eye_class=eyes_dict[box_eyes.cls.cpu().numpy().astype(int)[0]]
                    if eye_class=='awake':
                        eye_number=0
                    else:
                        eye_number=1
                    # x0,y0,x1,y1=box_eyes.xyxy.cpu().numpy().astype(int)[0]
                    xc_eyes,yc_eyes,width_bbox_eyes,height_bbox_eyes=box_eyes.xywhn.cpu().numpy()[0]
                    label_string_eyes=f'{eye_number} {xc_eyes} {yc_eyes} {width_bbox_eyes} {height_bbox_eyes}\n'
                    with open(label_path,'w') as f:
                        f.write(label_string_eyes) 
                image_name=common_name+'.png'
                image_path=os.path.join(save_image_folder,image_name)
               
                cv2.imwrite(image_path,frame2)
                # Saving label .txt file to a label folder
                xc,yc,width_bbox,height_bbox=result.boxes.xywhn.cpu().numpy()[0]
                label_string_yawn=f'{yawn_number} {xc} {yc} {width_bbox} {height_bbox}\n'
                with open(label_path,'a') as f:
                    f.write(label_string_yawn) 
                f.close()
        else:
            if yawn_AR>1.0:
                eyes_results=model_eyes.predict(frame2)
                eyes_result=eyes_results[0]
                video_name=path_yawn.split('/')[-1]
                frame_number=cap.get(cv2.CAP_PROP_POS_FRAMES)
                common_name=f'{video_name}_{frame_number}'
                label_name=common_name+'.txt'
                label_path=os.path.join(save_label_folder,label_name)

                for box_eyes in eyes_result.boxes:
                    eye_class=eyes_dict[box_eyes.cls.cpu().numpy().astype(int)[0]]
                    if eye_class=='awake':
                        eye_number=0
                    else:
                        eye_number=1
                    # x0,y0,x1,y1=box_eyes.xyxy.cpu().numpy().astype(int)[0]
                    xc_eyes,yc_eyes,width_bbox_eyes,height_bbox_eyes=box_eyes.xywhn.cpu().numpy()[0]
                    label_string_eyes=f'{eye_number} {xc_eyes} {yc_eyes} {width_bbox_eyes} {height_bbox_eyes}\n'
                    with open(label_path,'w') as f:
                        f.write(label_string_eyes) 
                
                image_name=common_name+'.png'
                image_path=os.path.join(save_image_folder,image_name)
               
                cv2.imwrite(image_path,frame2)
                # Saving label .txt file to a label folder
                
                xc,yc,width_bbox,height_bbox=result.boxes.xywhn.cpu().numpy()[0]
                label_string_yawn=f'{yawn_number} {xc} {yc} {width_bbox} {height_bbox}\n'
                with open(label_path,'a') as f:
                    f.write(label_string_yawn) 
                f.close()
                

        cv2.putText(frame,f"Yawn Ratio:{yawn_AR}",(20,50),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,0),2)
    
    cv2.imshow('test',frame)
    
    if cv2.waitKey(1) & 0xFF==27:
        print("Quitting the program")
        break
cap.release()
cv2.destroyAllWindows()



#### Generating the yawn image folder and label .txt

In [1]:
import numpy as np
def yawn_aspect_ratio_SPIGA(landmarks_int):
    """
    AR Formula= AR=||P2-P6||+||P3-P5||/2*(||P2-P4||)
    """
    
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    denominator_YAWN= np.linalg.norm(landmarks_int[91] - landmarks_int[88])  # Left Eye
    
    # ||P3-P5|| left
    yawn_89_95_dist=np.linalg.norm(landmarks_int[89] - landmarks_int[95])
   
    #||P2-P6|| left
    yawn_90_93_dist=np.linalg.norm(landmarks_int[90] - landmarks_int[93])
    
    yawn_AR=((yawn_89_95_dist+yawn_90_93_dist)/(2*denominator_YAWN))
    
    return yawn_AR

In [ ]:
import os
from ultralytics import YOLO
import numpy as np
import cv2

from spiga.demo.visualize.plotter import Plotter
from spiga.inference.config import ModelConfig
from spiga.inference.framework import SPIGAFramework

yawn_number=3   # The class label which will appear in label.txt

os.makedirs('yawnDD_yolo',exist_ok=True)
subfolder=['no_yawn','yawn']
sub_subfolder=['images','labels']
for folder in subfolder:
    os.makedirs(f'yawnDD_yolo/{folder}',exist_ok=True)
    for sub in sub_subfolder:
        os.makedirs(os.path.join(f'yawnDD_yolo/{folder}/{sub}'),exist_ok=True)

save_image_folder='yawnDD_yolo/yawn/images'
save_label_folder='yawnDD_yolo/yawn/labels'

# Importing the YOLO models
model_yolo_path=r'/home/cgi/drive/drowsy_att/YawDD.rar/yolov8n-face.pt'
model_eyes_path=r'/home/cgi/drive/drowsy_att/YawDD.rar/eye_detect.pt'
model_yolo=YOLO(model_yolo_path)
model_eyes=YOLO(model_eyes_path)

# Importing the SPIGA models
eyes_dict=model_eyes.names
dataset='wflw'
processor=SPIGAFramework(ModelConfig(dataset))
plotter=Plotter()

video_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/YAWN/videos/yawn'
video_list=os.listdir(video_folder)
for video in video_list:
    video_path=os.path.join(video_folder,video)
    cap=cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret,frame=cap.read()
        if not ret:
            print("End of Video")
            break
        frame2=frame.copy()
        results=model_yolo.predict(frame,conf=0.6)
        result=results[0]
        # Saving image file into the images folder

        for box in result.boxes:
            x0,y0,x1,y1=box.xyxy.cpu().numpy().astype(int)[0]
            cv2.rectangle(frame,(x0,y0),(x1,y1),(255,0,0),2)
            w=abs(x1-x0)
            h=abs(y1-y0)
            bbox=[x0,y0,w,h]
            features=processor.inference(frame,[bbox])
            # Prepare variables
            landmarks=np.array(features['landmarks'][0])
            headpose=np.array(features['headpose'][0])
            x1=round(headpose[0],3)
            x2=round(headpose[1],3)
            x3=round(headpose[2],3)
        
            # # print(headpose)
            frame=plotter.landmarks.draw_landmarks(frame,landmarks,thick=1)
            frame=plotter.hpose.draw_headpose(frame,[x0,y0,x0+w,y0+h],headpose[:3],headpose[3:],euler=True)
            landmarks_int=[landmark.astype('int') for landmark in landmarks]
            yawn_AR=yawn_aspect_ratio_SPIGA(landmarks_int)
            if 'Mirror' in video_path:
                if yawn_AR>0.5:
                    eyes_results=model_eyes.predict(frame2)
                    eyes_result=eyes_results[0]
                    video_name=video
                    frame_number=cap.get(cv2.CAP_PROP_POS_FRAMES)
                    common_name=f'{video_name}_{frame_number}'
                    label_name=common_name+'.txt'
                    label_path=os.path.join(save_label_folder,label_name)

                    for box_eyes in eyes_result.boxes:
                        eye_class=eyes_dict[box_eyes.cls.cpu().numpy().astype(int)[0]]
                        if eye_class=='awake':
                            eye_number=0
                        else:
                            eye_number=1
                        # x0,y0,x1,y1=box_eyes.xyxy.cpu().numpy().astype(int)[0]
                        xc_eyes,yc_eyes,width_bbox_eyes,height_bbox_eyes=box_eyes.xywhn.cpu().numpy()[0]
                        label_string_eyes=f'{eye_number} {xc_eyes} {yc_eyes} {width_bbox_eyes} {height_bbox_eyes}\n'
                        with open(label_path,'w') as f:
                            f.write(label_string_eyes) 
                    image_name=common_name+'.png'
                    image_path=os.path.join(save_image_folder,image_name)
                
                    cv2.imwrite(image_path,frame2)
                    # Saving label .txt file to a label folder
                    xc,yc,width_bbox,height_bbox=result.boxes.xywhn.cpu().numpy()[0]
                    label_string_yawn=f'{yawn_number} {xc} {yc} {width_bbox} {height_bbox}\n'
                    with open(label_path,'a') as f:
                        f.write(label_string_yawn) 
                    f.close()
            else:
                if yawn_AR>1.0:
                    eyes_results=model_eyes.predict(frame2)
                    eyes_result=eyes_results[0]
                    video_name=video
                    frame_number=cap.get(cv2.CAP_PROP_POS_FRAMES)
                    common_name=f'{video_name}_{frame_number}'
                    label_name=common_name+'.txt'
                    label_path=os.path.join(save_label_folder,label_name)

                    for box_eyes in eyes_result.boxes:
                        eye_class=eyes_dict[box_eyes.cls.cpu().numpy().astype(int)[0]]
                        if eye_class=='awake':
                            eye_number=0
                        else:
                            eye_number=1
                        # x0,y0,x1,y1=box_eyes.xyxy.cpu().numpy().astype(int)[0]
                        xc_eyes,yc_eyes,width_bbox_eyes,height_bbox_eyes=box_eyes.xywhn.cpu().numpy()[0]
                        label_string_eyes=f'{eye_number} {xc_eyes} {yc_eyes} {width_bbox_eyes} {height_bbox_eyes}\n'
                        with open(label_path,'w') as f:
                            f.write(label_string_eyes) 
                    
                    image_name=common_name+'.png'
                    image_path=os.path.join(save_image_folder,image_name)
                
                    cv2.imwrite(image_path,frame2)
                    # Saving label .txt file to a label folder
                    
                    xc,yc,width_bbox,height_bbox=result.boxes.xywhn.cpu().numpy()[0]
                    label_string_yawn=f'{yawn_number} {xc} {yc} {width_bbox} {height_bbox}\n'
                    with open(label_path,'a') as f:
                        f.write(label_string_yawn) 
                    f.close()
                    

            cv2.putText(frame,f"Yawn Ratio:{yawn_AR}",(20,50),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,0),2)
        
        cv2.imshow('test',frame)
        
        if cv2.waitKey(1) & 0xFF==27:
            print("Quitting the program")
            break
    cap.release()
cv2.destroyAllWindows()




#### Saving the talking images

Take 75 frames from each video

In [ ]:
import os
import cv2
import random
from ultralytics import YOLO
save_image_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/yawnDD_yolo/no_yawn/images'
save_label_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/yawnDD_yolo/no_yawn/labels'

# Importing the YOLO models
model_yolo_path=r'/home/cgi/drive/drowsy_att/YawDD.rar/yolov8n-face.pt'
model_eyes_path=r'/home/cgi/drive/drowsy_att/YawDD.rar/eye_detect.pt'
model_yolo=YOLO(model_yolo_path)
model_eyes=YOLO(model_eyes_path)

eyes_dict=model_eyes.names
no_yawn_number=2

# Folder where the no_yawn videos are saved
video_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/YAWN/videos/no_yawn'
video_list=os.listdir(video_folder)

for video in video_list:
    frame_count=0
    random_list=[]
    video_path=os.path.join(video_folder,video)
    cap=cv2.VideoCapture(video_path)
    total_frames=cap.get(cv2.CAP_PROP_FRAME_COUNT)
    while frame_count<75:
        random_int=random.randint(0,total_frames-1)
        while random_int in random_list:
            random_int=random.randint(0,total_frames-1)
        
        save_image_name=f'{video}_{random_int}.png'
        save_label_name=f'{video}_{random_int}.txt'
        # Creating the save paths for the images
        save_image_path=os.path.join(save_image_folder,save_image_name)
        save_label_path=os.path.join(save_label_folder,save_label_name)  

        cap.set(cv2.CAP_PROP_POS_FRAMES,random_int)
        random_list.append(random_int)
        ret,frame=cap.read()
        label_list=[]
        frame2=frame.copy()
        
        results=model_yolo.predict(frame,conf=0.6)
        eyes_results=model_eyes.predict(frame,conf=0.6)
        eyes_result=eyes_results[0]
        result=results[0]

        for box in result.boxes:
            x0,y0,x1,y1=box.xyxy.cpu().numpy().astype(int)[0]
            xc,yc,width_bbox,height_bbox=result.boxes.xywhn.cpu().numpy()[0]
            label_string_yawn=f'{no_yawn_number} {xc} {yc} {width_bbox} {height_bbox}\n'
            label_list.append(label_string_yawn)
            

            cv2.rectangle(frame,(x0,y0),(x1,y1),(255,0,0),2)
        for box_eyes in eyes_result.boxes:
            eye_class=eyes_dict[box_eyes.cls.cpu().numpy().astype(int)[0]]
            if eye_class=='awake':
                eye_number=0
            else:
                eye_number=1
            x0,y0,x1,y1=box.xyxy.cpu().numpy().astype(int)[0]
            xc_eyes,yc_eyes,width_bbox_eyes,height_bbox_eyes=box_eyes.xywhn.cpu().numpy()[0]
            label_string_eyes=f'{eye_number} {xc_eyes} {yc_eyes} {width_bbox_eyes} {height_bbox_eyes}\n'
            cv2.rectangle(frame,(x0,y0),(x1,y1),(255,0,0),2)
            cv2.putText(frame,eye_class,(x0-10,y0-20),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,255),2)
            label_list.append(label_string_eyes)

        cv2.imshow('no_yawn',frame)
        if cv2.waitKey(1) & 0xff==27:
            print('Quitting the program')
            break
        with open(save_label_path,'w') as f:
            f.writelines(label_list)
        f.close()
        cv2.imwrite(save_image_path,frame2)
        frame_count+=1
    
    cap.release()
    # for box in result.boxes:
    #     x0,y0,x1,y1=box.xyxy.cpu().numpy().astype(int)[0]
    #     cv2.rectangle(frame,(x0,y0),(x1,y1),(255,0,0),2)
    # cv2.imshow('test',frame)
    # cv2.waitKey(0)
cv2.destroyAllWindows()
    

In [ ]:
cv2.destroyAllWindows()

In [124]:
model_eyes=YOLO('/home/cgi/drive/drowsy_att/YawDD.rar/eye_detect.pt')
test_results=model_eyes.predict(frame2)


0: 480x640 1 awake, 2.7ms
Speed: 0.5ms preprocess, 2.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)


In [ ]:
test_eyes=frame2.copy()
eyes_dict=model_eyes.names
test_result=test_results[0]
for box_eyes in test_result.boxes:
    eye_class=eyes_dict[box_eyes.cls.cpu().numpy().astype(int)[0]]
    x0,y0,x1,y1=box_eyes.xyxy.cpu().numpy().astype(int)[0]
    xc_eyes,yc_eyes,width_bbox_eyes,height_bbox_eyes=box_eyes.xywhn.cpu().numpy().astype(int)[0]
    cv2.rectangle(test_eyes,(x0,y0),(x1,y1),(0,0,255),2)

    print(x0,y0,x1,y1)
    print(box_eyes.xyxy.cpu().numpy().astype(int)[0])
    print(box_eyes)
cv2.imshow('eyes',test_eyes)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Making a train and val dataset from the resulting folder

3500 images in train 625 in val

In [12]:
import os
import random
import shutil
# Making the nesting folders to save the final result in 
os.makedirs('final_dataset',exist_ok=True)
folder_list=['train','valid']
subfolder_list=['images','labels']
for folder in folder_list:
    folder_path=os.path.join('final_dataset',folder)
    os.makedirs(folder_path,exist_ok=True)
    for subfolder in subfolder_list:
        subfolder_path=os.path.join(folder_path,subfolder)
        os.makedirs(subfolder_path,exist_ok=True)

yawn_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/yawnDD_yolo/yawn'
no_yawn_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/yawnDD_yolo/no_yawn'

yawn_image_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/yawnDD_yolo/yawn/images'
no_yawn_image_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/yawnDD_yolo/no_yawn/images'

yawn_label_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/yawnDD_yolo/yawn/labels'
no_yawn_label_folder=r'/home/cgi/drive/drowsy_att/YawDD.rar/yawnDD_yolo/no_yawn/labels'

yawn_list=os.listdir(yawn_image_folder)
no_yawn_list=os.listdir(no_yawn_image_folder)

yawn_image_count=0
no_yawn_image_count=0
# For yawn_list
random_list=[]
while yawn_image_count<4125:
    random_int=random.randint(0,len(yawn_list)-1)
    while random_int in random_list:
        random_int=random.randint(0,len(yawn_list)-1)
    random_list.append(random_int)
    image=yawn_list[random_int]
    image_name=image.split('.png')[0]
    label_name=image_name+'.txt'
    image_file_path=os.path.join(yawn_image_folder,image)
    label_file_path=os.path.join(yawn_label_folder,label_name)
    if yawn_image_count<3500:
        dest_image_folder='./final_dataset/train/images'
        dest_label_folder='./final_dataset/train/labels'
        dest_image_path=os.path.join(dest_image_folder,image)
        dest_label_path=os.path.join(dest_label_folder,label_name)
        # Copy the images and file
        # shutil.copy(source_file, destination_file)
        shutil.copy(label_file_path,dest_label_path)
        shutil.copy(image_file_path,dest_image_path)
        yawn_image_count+=1
    else:
        dest_image_folder='./final_dataset/valid/images'
        dest_label_folder='./final_dataset/valid/labels'
        dest_image_path=os.path.join(dest_image_folder,image)
        dest_label_path=os.path.join(dest_label_folder,label_name)
        # Copy the images and file
        # shutil.copy(source_file, destination_file)
        shutil.copy(label_file_path,dest_label_path)
        shutil.copy(image_file_path,dest_image_path)
        yawn_image_count+=1

# For the no_yawn list
random_list=[]
while no_yawn_image_count<4125:
    random_int=random.randint(0,len(no_yawn_list)-1)
    while random_int in random_list:
        random_int=random.randint(0,len(no_yawn_list)-1)
    random_list.append(random_int)
    image=no_yawn_list[random_int]
    image_name=image.split('.png')[0]
    label_name=image_name+'.txt'
    image_file_path=os.path.join(no_yawn_image_folder,image)
    label_file_path=os.path.join(no_yawn_label_folder,label_name)
    if no_yawn_image_count<3500:
        dest_image_folder='./final_dataset/train/images'
        dest_label_folder='./final_dataset/train/labels'
        dest_image_path=os.path.join(dest_image_folder,image)
        dest_label_path=os.path.join(dest_label_folder,label_name)
        # Copy the images and file
        # shutil.copy(source_file, destination_file)
        shutil.copy(label_file_path,dest_label_path)
        shutil.copy(image_file_path,dest_image_path)
        no_yawn_image_count+=1
    else:
        dest_image_folder='./final_dataset/valid/images'
        dest_label_folder='./final_dataset/valid/labels'
        dest_image_path=os.path.join(dest_image_folder,image)
        dest_label_path=os.path.join(dest_label_folder,label_name)
        # Copy the images and file
        # shutil.copy(source_file, destination_file)
        shutil.copy(label_file_path,dest_label_path)
        shutil.copy(image_file_path,dest_image_path)
        no_yawn_image_count+=1

#### Merging with the two yolo datasets

In [44]:
import shutil
HOME='final_dataset'
folder_list=os.listdir(HOME)
for folder in folder_list:
    if folder=='train':
        image_folder='final_dataset/train/images'
        label_folder='final_dataset/train/labels'
        image_list=os.listdir(image_folder)
        label_list=os.listdir(label_folder)
        for image in image_list:
            image_path=os.path.join(image_folder,image)
            dst_path=f'/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness Detection.v5i.yolov8/train/images/{image}'
            shutil.copy(image_path,dst_path)
        for label in label_list:
            label_path=os.path.join(label_folder,label)
            dst_label_path=f'/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness Detection.v5i.yolov8/train/labels/{label}'
            shutil.copy(label_path,dst_label_path)
        
    else:
        image_folder='final_dataset/valid/images'
        label_folder='final_dataset/valid/labels'
        image_list=os.listdir(image_folder)
        label_list=os.listdir(label_folder)
        for image in image_list:
            image_path=os.path.join(image_folder,image)
            dst_path=f'/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness Detection.v5i.yolov8/valid/images/{image}'
            shutil.copy(image_path,dst_path)
        for label in label_list:
            label_path=os.path.join(label_folder,label)
            dst_label_path=f'/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness Detection.v5i.yolov8/valid/labels/{label}'
            shutil.copy(label_path,dst_label_path)


In [43]:
import shutil
HOME='final_dataset'
folder_list=os.listdir(HOME)
for folder in folder_list:
    if folder=='train':
        image_folder='final_dataset/train/images'
        label_folder='final_dataset/train/labels'
        image_list=os.listdir(image_folder)
        label_list=os.listdir(label_folder)
        for image in image_list:
            image_path=os.path.join(image_folder,image)
            dst_path=f'/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness Detection.v5i.yolov8/train/images/{image}'
            shutil.copy(image_path,dst_path)
            break
        for label in label_list:
            label_path=os.path.join(label_folder,label)
            dst_label_path=f'/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness Detection.v5i.yolov8/train/labels/{label}'
            shutil.copy(label_path,dst_label_path)
            break

In [41]:
label

'Mirror_8-MaleGlassesBeard-Yawning.avi_222.0.txt'

In [40]:
label_path

'final_dataset/train/labels/Mirror_8-MaleGlassesBeard-Yawning.avi_222.0.txt'

In [37]:
dst_path

'/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness Detection.v5i.yolov8/train/images/Mirror_42-FemaleSunGlasses-Talking.avi_181.png'

In [35]:
dst__label_path

'/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness Detection.v5i.yolov8/valid/labels/Mirror_1-FemaleNoGlasses-Talking.avi_526.txt'

Take 0.7 as threshold of yawning for the mirror videos and 0.9 as threshold for the front videos(videos that do not have mirror in their name)

76-95 Lips landmark 

Checking if classes 3 and 4 are in labels

In [45]:
path='/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness Detection.v5i.yolov8/train/labels'
# path=r'/home/cgi/drive/drowsy_att/YawDD.rar/Drowsiness/train/labels'
label_list=os.listdir(path)
no_yawn=0
yawn=0
for label in label_list:
    label_path=os.path.join(path,label)
    with open(label_path,'r') as f:
        contents=f.readlines()
        for content in contents:
            label_line=content.strip('\n')
            label_number=label_line.split(' ')[0]
            if label_number=='2':
                no_yawn+=1
            elif label_number=='3':
                yawn+=1


In [46]:
no_yawn,yawn

(3200, 3501)

In [23]:
no_yawn,yawn

(3200, 3501)